In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

# bfs4
from bs4 import BeautifulSoup
import urllib.request
import requests

import pandas as pd

In [140]:
df_temp=pd.read_csv('Tanitjobs.csv')

In [5]:
options =webdriver.ChromeOptions()
options.headless=False
prefs={"profile.default_content_setting_values.notifications" :2}
options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome('C:/chromedriver/chromedriver.exe')

driver.get("https://www.tanitjobs.com/")
driver.find_element_by_xpath('//*[@id="keywords"]').send_keys("informatique",Keys.ENTER)
time.sleep(5)

num_links = len(driver.find_elements_by_class_name('pad_right_small'))
num_links

for i in range(50):
    driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[3]/div[4]/button").click()

# num des annonces qui se trouvent dans la page apres le click dde voir plus
for i in range(len(driver.find_elements_by_class_name("link"))):
    element = driver.find_elements_by_class_name("link")[i]
    ActionChains(driver) \
        .key_down(Keys.CONTROL) \
        .click(element) \
        .key_up(Keys.CONTROL) \
        .perform()       
    

list_url=[]
##If you have more than one switch to other window
if len(driver.window_handles) > 1:
    for i in range(len(driver.find_elements_by_class_name("link"))-1):
        driver.switch_to.window(driver.window_handles[i])
        print(driver.current_url)
        list_url.append(driver.current_url)

ElementClickInterceptedException: Message: element click intercepted: Element <button type="button" class="load-more btn btn__nextpage" data-page="2" data-backfilling="false" data-backfilling-page="1">...</button> is not clickable at point (630, 654). Other element would receive the click: <article id="651214" class="media well listing-item listing-item__jobs  ">...</article>
  (Session info: chrome=83.0.4103.61)


In [147]:
list_url.remove(list_url[0])
list_url.remove(list_url[len(list_url)-1])

In [148]:
list_url[len(list_url)-1]

'https://www.tanitjobs.com/job/566968/des-conseillers-clients-fran%C3%A7ais/?backPage=1&searchID=1591477134.2972'

In [149]:
len(list_url)

59

## BFSoup 

In [153]:
zebi=0
for i in range(len(list_url)):
    urlpage = list_url[i]
    page = urllib.request.urlopen(urlpage)
    soup = BeautifulSoup(page, 'html.parser')

    description =soup.find_all('div', class_='details-body__content content-text')[0].get_text()
    description = description.replace(u'\xa0', u' ')
    description = description.replace(u'\n', u' ')

    uls = soup.select("div.bootstrap-tagsinput")
    text = [a.text for ul in uls for a in  ul.select("a")]
#------------------------------------------------------------------------------------

    df = pd.DataFrame({'description':[description],'Tags':[text]})
    df = pd.concat([df, df_temp])
    #Delet index
    df.reset_index(drop=True, inplace=True)

    df.to_csv('Tanitjobs.csv',index=False)

    zebi+=1    

In [151]:
zebi

59

# Etape 1

In [14]:
df_temp=pd.read_csv('Tanitjobs.csv')

page = urllib.request.urlopen("https://www.tanitjobs.com/job/596558/technicien-superieur-en-informatique/?backPage=9&searchID=1591546268.7353")

# Analyze the html using beautiful soup
soup = BeautifulSoup(page, 'html.parser')


description =soup.find_all('div', class_='details-body__content content-text')[1].get_text()
description = description.replace(u'\xa0', u' ')
description = description.replace(u'\n', u' ')

uls = soup.select("div.bootstrap-tagsinput")
text = [a.text for ul in uls for a in  ul.select("a")]  

df = pd.DataFrame({'description':[description],'Tags':[text]})
df = pd.concat([df, df_temp])
#Delet index
df.reset_index(drop=True, inplace=True)
df.to_csv('Tanitjobs.csv',index=False)

In [15]:
dfg.head()

,description,Tags
0,"Exigences de l'emploi Vous êtes sérieux, moti...","['Formateur ', 'Développement ', 'Informatiq..."
1,Profil:-De formation supérieure (BAC+3 a plus)...,"['Community Manager ', 'Marketing Digitale ', ..."
2,Vous êtes d'une formation minimum Bac+3/5 ave...,"['Développeur ', 'Informatique ', 'Web ', '..."
3,"Compétences requises - React Native, MongoDB, ...","['Développeur ', 'Mobile ', 'Informatique '..."
4,Profil demandé : Un français courant exigé Une...,"['Téléconseillère ', 'Call Center ', ""Centre ..."


In [11]:
df.drop_duplicates(subset=['description'],keep='last')

,description,Tags
0,Compétences techniques : SQL Server/requêtes S...,"[Responsable Applicatif , Informatique , SQL..."
1,"Au sein de notre équipe R&D, vous participerez...","['Ingénieur ', 'Informatique ', 'Développeme..."
2,"Exigences de l'emploi Vous êtes sérieux, moti...","['Formateur ', 'Développement ', 'Informatiq..."
3,Profil:-De formation supérieure (BAC+3 a plus)...,"['Community Manager ', 'Marketing Digitale ', ..."
4,Vous êtes d'une formation minimum Bac+3/5 ave...,"['Développeur ', 'Informatique ', 'Web ', '..."
...,...,...
91,Votre profil pour mener à bien cette mission ...,['Web/Chef de projet/Suivi stratégique ']
92,Issu(e) d’une formation supérieur en communic...,['Community manager rédacteur de contenu ']
93,06/07/2020,[]
94,Ability to work in a team. Degree in Computer...,"['php ', 'css ', 'jquery ', 'js ', 'reactjs ',..."


In [20]:
from os import path
from googletrans import Translator

translator = Translator()
trans = [] 
for text in df.description:
    if translator.detect(text).lang == 'fr':
        trans.append(translator.translate(text).text)
    else:
        trans.append(text)

In [23]:
print(trans)

 test cycle. You are familiar with the test repository management tools. You are familiar with the tools and issues of automated testing You are familiar with the API test and programming Python French, clear and precise expression, easy to synthesize and disseminate information. You enjoy teamwork It's even better if: You have a good understanding of web technologies HTML / CSS / Javascript / vue.Js / API you have ideas on agile methodologies At NeoXam, we love the curious people involved and autonomous, who love to provide new solutions and for which it is essential to continue to learn and grow every day. The job requires great rigor and toughness, ability to communicate and promote good practice. WE OFFER A permanent position dynamic work environment and challengeant. A competitive package consisting of: fixed + individual bonus And much more ...", 'Your skills: Good experience with Ionic Peculiar architectural patterns Peculiar with REST / SOAP / WEBSERVICES Peculiar Git SCRUM SQL

In [22]:
len(trans)

98

In [27]:
df['text']=trans

In [25]:
# add trnaslate list in our dataframe
df2 = pd.DataFrame(data=trans, columns={"Translated"})

In [33]:
df=df.drop(columns={'description'})
df=df.drop(columns={'transated'})

In [35]:
df=df[['text','Tags']]

In [38]:
df.head()

,text,Tags
0,Skills and Qualifications Bachelor's degree i...,"[informatique , Technologie de l'information ,..."
1,Profile: You have completed your technical stu...,"['Développeur ', 'Informatique ', 'Web ', '..."
2,"Technical Skills: SQL Server / SQL, basic Sync...","['Responsable Applicatif ', 'Informatique ',..."
3,"In our R & D team, you will participate in the...","['Ingénieur ', 'Informatique ', 'Développeme..."
4,"Job requirements you serious, motivated, punct...","['Formateur ', 'Développement ', 'Informatiq..."


In [37]:
df.to_csv("Tanitjobs.csv")

# `*********************************`

In [4]:
options =webdriver.ChromeOptions()
options.headless=False
prefs={"profile.default_content_setting_values.notifications" :2}
options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome('C:/chromedriver/chromedriver.exe')

driver.get("https://www.tanitjobs.com/")
driver.find_element_by_xpath('//*[@id="keywords"]').send_keys("it",Keys.ENTER)
time.sleep(1)


num_links = len(driver.find_elements_by_class_name('pad_right_small'))
num_links

a=0
for i in range(50):
    driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[3]/div[4]/button").click()
    a+=1


for i in range(len(driver.find_elements_by_class_name("link"))):
    button = driver.find_elements_by_class_name("link")[i]
    button.click()
    print(driver.current_url)
    driver.execute_script("window.history.go(-1)")  

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=83.0.4103.61)


In [ ]:
list_url.remove(list_url[0])

In [ ]:
zebi=0
for i in range(len(list_url)):
    urlpage = list_url[i]
    page = urllib.request.urlopen(urlpage)
    soup = BeautifulSoup(page, 'html.parser')

    description =soup.find_all('div', class_='details-body__content content-text')[0].get_text()
    description = description.replace(u'\xa0', u' ')
    description = description.replace(u'\n', u' ')

    uls = soup.select("div.bootstrap-tagsinput")
    text = [a.text for ul in uls for a in  ul.select("a")]
#------------------------------------------------------------------------------------

    df = pd.DataFrame({'description':[description],'Tags':[text]})
    df = pd.concat([df, df_temp])
    #Delet index
    df.reset_index(drop=True, inplace=True)

    df.to_csv('Tanitjobs.csv',index=False)

    zebi+=1    

In [39]:
df=pd.read_csv('ideed_Eng.csv')

In [40]:
df.head()

,entreprise,titre,description
0,The Standard,Senior Data Scientist,"At The Standard, you’ll join a team focused on..."
1,dunnhumby,Senior Applied Data Scientist,dunnhumby is the world’s leading customer scie...
2,"HEPCO, Inc.",Senior Data Scientist,"SENIOR DATA SCIENTIST HEPCO, Inc., is seeking..."
3,One Concern,"Data Scientist, Machine Learning",One Concern is a startup which specializes in ...
4,"Grammarly, Inc.",Data Scientist,The opportunity Grammarly empowers people to ...


In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [67]:
class SkillsExtractorNN:

    def __init__(self, word_features_dim, dense_features_dim):

        lstm_input_phrase = keras.layers.Input(shape=(None, word_features_dim))
        lstm_input_cont = keras.layers.Input(shape=(None, word_features_dim))
        dense_input = keras.layers.Input(shape=(dense_features_dim,))

        lstm_emb_phrase = keras.layers.LSTM(256)(lstm_input_phrase)
        lstm_emb_phrase = keras.layers.Dense(128, activation='relu')(lstm_emb_phrase)

        lstm_emb_cont = keras.layers.LSTM(256)(lstm_input_cont)
        lstm_emb_cont = keras.layers.Dense(128, activation='relu')(lstm_emb_cont)

        dense_emb = keras.layers.Dense(512, activation='relu')(dense_input)
        dense_emb = keras.layers.Dense(256, activation='relu')(dense_emb)

        x = keras.layers.concatenate([lstm_emb_phrase, lstm_emb_cont, dense_emb])
        x = keras.layers.Dense(128, activation='relu')(x)
        x = keras.layers.Dense(64, activation='relu')(x)
        x = keras.layers.Dense(32, activation='relu')(x)

        main_output = keras.layers.Dense(2, activation='softplus')(x)

        self.model = keras.models.Model(inputs=[lstm_input_phrase, lstm_input_cont, dense_input],
                                        outputs=main_output)

        optimizer = keras.optimizers.Adam(lr=0.0001)

        self.model.compile(optimizer=optimizer, loss='binary_crossentropy')

In [1]:
    def fit(self, x_lstm_phrase, x_lstm_context, x_dense, y,
            val_split=0.25, patience=5, max_epochs=1000, batch_size=32):

        x_lstm_phrase_seq = keras.preprocessing.sequence.pad_sequences(x_lstm_phrase)
        x_lstm_context_seq = keras.preprocessing.sequence.pad_sequences(x_lstm_context)

        y_onehot = onehot_transform(y)

        self.model.fit([x_lstm_phrase_seq, x_lstm_context_seq, x_dense],
                       y_onehot,
                       batch_size=batch_size,
                       pochs=max_epochs,
                       validation_split=val_split,
                       callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)])


    def predict(self, x_lstm_phrase, x_lstm_context, x_dense):

        x_lstm_phrase_seq = keras.preprocessing.sequence.pad_sequences(x_lstm_phrase)
        x_lstm_context_seq = keras.preprocessing.sequence.pad_sequences(x_lstm_context)

        y = self.model.predict([x_lstm_phrase_seq, x_lstm_context_seq, x_dense])

        return y

In [3]:
def onehot_transform(y):

    onehot_y = []

    for numb in y:
        onehot_arr = np.zeros(2)
        onehot_arr[numb] = 1
        onehot_y.append(np.array(onehot_arr))

    return np.array(onehot_y)